In [ ]:
import pandas as pd
import re
from datetime import datetime

# Задание на работу с массивом данных (Python)
Выполнил: Алексей Воронко

Основные нюансы работы скрипта:
1.	Результат должен быть точно таким же, как в файле «Результат обработки.xlsx»;
2.	Скрипт должен фильтровать по Московскому банку (сотрудники Московского банка указаны в файле «ШЕ.xlsx»;
3.	Скрипт определяет сегмент исходя из файла «ШЕ.xlsx»;
4.	Исходя из сегмента определяется Направление, где:  
- СБП и СБ1 = Премиальное направление  
- ВСП = Массовое направление  
-  КБ = Корпоративное направление;  
5. Скрипт должен оставить только проведённые активности – это те активности, где «Техническое поле 3» меньше «Время регистрации»;
6.	Скрипт должен фильтровать только ВСП с припиской «9038_»;
7.	Поле «Дата загрузки» проставляет автоматически текущую дату и время.


## Чтение данных

Прочитаем данные, выведем информацию о таблицах

In [ ]:
df_origin = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/sber_mos_bank/Исходный массив.xlsx')

In [ ]:
df_origin.sample()

,Техническое поле 10,Техническое поле 8,Техническое поле 7,Техническое поле 6,Техническое поле 5,Техническое поле 4,Техническое поле 3,Техническое поле 2,Техническое поле 1,Услуга,Статус,Время регистрации,Контрольный срок,Краткое описание
256,Премьер,ПАО Сбербанк/Территориальные банки/Поволжский ...,ППО,Очное обучение,Королева (458639),Обучение БФО,2020-07-31,Дополнительный офис №8621/0709,NaN,Поддержка вертикали блока Управление благосост...,Закрыто,2020-08-02 15:29:24,2020-08-03 13:00:00,Внесение активностей сотрудников массового выс...


In [ ]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Техническое поле 10  511 non-null    object        
 1   Техническое поле 8   511 non-null    object        
 2   Техническое поле 7   511 non-null    object        
 3   Техническое поле 6   511 non-null    object        
 4   Техническое поле 5   511 non-null    object        
 5   Техническое поле 4   511 non-null    object        
 6   Техническое поле 3   511 non-null    datetime64[ns]
 7   Техническое поле 2   329 non-null    object        
 8   Техническое поле 1   82 non-null     object        
 9   Услуга               511 non-null    object        
 10  Статус               511 non-null    object        
 11  Время регистрации    511 non-null    datetime64[ns]
 12  Контрольный срок     511 non-null    datetime64[ns]
 13  Краткое описание     511 non-null  

In [ ]:
df_example = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/sber_mos_bank/Результат обработки.xlsx')

In [ ]:
df_example.sample()

,Продукт/Услуга,Цель/Причина визита,Дата,ФИО коуча,Направление,Категория,Сегмент,ВСП,ТБ,Проведенные активности,Дата загрузки
87,Теория: макроэкономика/инвестиции/страхование,Обучение сотрудников УППБ,2020-08-20,Свинюхин,Премиальное направление,Проведённая активность,СБП,МБ/0013,Московский банк,Дистанционное обучение ВКС,2020-09-01 18:30:54.629


In [ ]:
df_example.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Продукт/Услуга          176 non-null    object        
 1   Цель/Причина визита     176 non-null    object        
 2   Дата                    176 non-null    datetime64[ns]
 3   ФИО коуча               176 non-null    object        
 4   Направление             176 non-null    object        
 5   Категория               176 non-null    object        
 6   Сегмент                 176 non-null    object        
 7   ВСП                     176 non-null    object        
 8   ТБ                      176 non-null    object        
 9   Проведенные активности  176 non-null    object        
 10  Дата загрузки           176 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(9)
memory usage: 15.2+ KB


In [ ]:
df_clients = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/sber_mos_bank/ШЕ.xlsx')

In [ ]:
df_clients

,Таб. Номер,Фамилия,Сегмент,ТБ
0,1386494,Иванов,КБ,Московский банк
1,1348033,Феротова,ВСП,Московский банк
2,31031,Скулакова,ВСП,Московский банк
3,1305104,Тяжелов,ВСП,Московский банк
4,1156848,Свинюхин,СБП,Московский банк
5,1385856,Галяутдинов,СБП,Московский банк
6,40313,Крикушин,СБП,Московский банк
7,1808036,Цквитария,СБ1,Московский банк
8,1386336,Амиянц,СБП,Московский банк
9,1354986,Таранец,КБ,Московский банк


## Проработка по условию задачи

### Переименование столбцов

In [ ]:
df_origin.columns = ['Техническое поле 10', 'ВСП', 'Продукт/Услуга',
       'Проведенные активности', 'ФИО коуча', 'Цель/Причина визита',
       'Дата', 'Техническое поле 2', 'Техническое поле 1',
       'Услуга', 'Статус', 'Время регистрации', 'Контрольный срок',
       'Краткое описание']

### Удаление символов в ФИО коуча

In [ ]:
def only_name(row):
  text = re.sub(r'[^а-яА-ЯёЁ ]', ' ', row)
  text = text.split()
  text = " ".join(text)
  return text

df_origin['ФИО коуча'] = df_origin['ФИО коуча'].apply(only_name)

### Только проведенные активности

Согласно условию № 5: Скрипт должен оставить только проведённые активности – это те активности, где «Техническое поле 3» меньше «Время регистрации»

In [ ]:
df_origin = df_origin[df_origin['Дата'] < df_origin['Время регистрации']]

In [ ]:
df_origin['Категория'] = 'Проведённая активность'

### Определим МБ (приписка "9038_")

Согласно условию № 6:	Скрипт должен фильтровать только ВСП с припиской «9038_»

In [ ]:
df_origin = df_origin[df_origin['ВСП'].str.contains('9038_').fillna(False)]

In [ ]:
def change_id_bank(row):
  return row.replace('9038_', 'МБ/')

df_origin['ВСП'] = df_origin['ВСП'].apply(change_id_bank)

### Фильтр по Московскому банку

Согласно условию № 2:	Скрипт должен фильтровать по Московскому банку (сотрудники Московского банка указаны в файле «ШЕ.xlsx»   
Согласно условию № 3:	Скрипт определяет сегмент исходя из файла «ШЕ.xlsx»;

In [ ]:
df_origin = df_origin.merge(df_clients, left_on='ФИО коуча', right_on='Фамилия')

### Определение направления

Согласно условию № 4:	Исходя из сегмента определяется Направление, где:  
СБП и СБ1 = Премиальное направление  
ВСП = Массовое направление  
КБ = Корпоративное направление

In [ ]:
def bank_direction(row):
  if row == 'ВСП':
    return 'Массовое направление'
  elif row == 'КБ':
    return 'Корпоративное направление'
  else:
    return 'Премиальное направление'

In [ ]:
df_origin['Направление'] = df_origin['Сегмент'].apply(bank_direction)

### Столбец Дата загрузки

Согласно условию № 7: Поле «Дата загрузки» проставляет автоматически текущую дату и время

In [ ]:
current_datetime = datetime.now().replace(microsecond=0)

In [ ]:
df_origin['Дата загрузки'] = current_datetime

## Выгрузка готового (переработанного) массива

Переопределим следование столбцов:

In [ ]:
df_origin = df_origin[['Продукт/Услуга', 'Цель/Причина визита', 'Дата', 'ФИО коуча',
 'Направление', 'Категория', 'Сегмент', 'ВСП', 'ТБ', 'Проведенные активности', 'Дата загрузки']]

Поскольку задание "Результат должен быть точно таким же, как в файле «Результат обработки.xlsx»", нужно получить отчет за август 2020. Значит удалим даты младше 01  августа 2020 года.

In [ ]:
df_origin = df_origin[df_origin['Дата'] > "2020-08-01 00:00:00"]

Проведем сортировку (до конца так и не понял, как выполнена сортировка в примере):

In [ ]:
df_origin = df_origin.sort_values(by=['Дата', 'Направление',])

### Выгрузка файла

In [ ]:
df_origin.to_excel("Результат обработки.xlsx", index=False,
             sheet_name='Лист 1')

In [ ]:
# df_origin.head()